# $\color{#FF3399}{\textbf{Mini-project: Contact Management System (CRUD) ☎️}}$

$\color{#FF66B2}{\textbf{Task description}}$

**Model**: a contact is a simple dictionary: `{"first_name": str, "last_name": str, "email": str}`.

**Storage**: `store: dict[email] -> contact`.

**Operations**: `create`, `read/list`, `find_by_lastname`, `update`, `delete`.

**Minimal validation**: trim leading and trailing spaces in all fields, email must contain @.

$\color{#FF66B2}{\textbf{Helper functions}}$

In [4]:
def _norm_name(s: str) -> str:
    """Trim spaces and make the first letter uppercase (rest is lowercase)."""
    return s.strip().casefold().title()

def _valid_email(email: str) -> bool:
    """Check if email contains something '@' sign and something after '@' sign (without regex")."""
    return '@' in email and email[email.index("@") + 1:] != ""

$\color{#FF66B2}{\textbf{CRUD}}$

In [8]:
def create_contact(store: dict[str, dict[str, str]], raw: dict[str, str]) -> None:
    """
    Validate, format and add a contact to the store.

    Args:
        store (dict[str, dict[str, str]]): Dictionary where contacts are stored (email → contact).
        raw (dict[str, str]): Raw contact data with keys 'first_name', 'last_name', and 'email'.

    Raises:
        KeyError: If one of the required keys is missing.
        ValueError: If the email is invalid or already exists in the store.
    """
    try:
        name, surname = _norm_name(raw["first_name"]), _norm_name(raw["last_name"]) 
        email = raw["email"].strip()

        if not _valid_email(email):
            raise ValueError("Email must contain @ and something after this sign")

        if email in store.keys():
            raise ValueError("This email is a duplicate")

        store[email] = {
            "first_name": name,
            "last_name": surname,
            "email": email
        }

    except KeyError:
        raise KeyError
    except ValueError:
        raise

def list_contacts(store: dict[str, dict[str, str]]) -> list[dict[str, str]]:
    """Return a list of all contacts in the order they were added."""
    return list(store.values())

def find_by_lastname(store: dict[str, dict[str, str]], fragment: str) -> list[dict[str, str]]:
    """Find all contacts whose last name contains the given fragment (case-insensitive)."""
    return [contact for contact in store.values() if fragment.casefold() in contact["last_name"].casefold()] 

def update_contact(store: dict[str, dict[str, str]], email: str, **changes) -> dict[str, str]:
    """
    Update a contact's first or last name (email cannot be changed).

    Args:
        store (dict[str, dict[str, str]]): Dictionary where contacts are stored.
        email (str): Email of the contact to update.
        **changes: Keyword arguments with 'first_name' and/or 'last_name' fields to update.

    Returns:
        dict[str, str]: The updated contact record.

    Raises:
        KeyError: If no contact exists with the given email.
        ValueError: If provided data is invalid.
    """
    try:
        c = store[email]

        if "first_name" in changes:
            store[email]["first_name"] = _norm_name(changes["first_name"])

        if "last_name" in changes:
            store[email]["last_name"] = _norm_name(changes["last_name"])

        return c

    except KeyError:
        raise
    except ValueError:
        raise

def delete_contact(store: dict[str, dict[str, str]], email: str) -> bool:
    """Delete last contact - returns True if the contact existed and was deleted, False otherwise."""
    return store.pop(email, None) is not None

$\color{#FF66B2}{\textbf{Basic tests and error tests}}$

In [9]:
_store: dict[str, dict[str, str]] = {}

# CREATE
create_contact(_store, {"first_name": " ala ", "last_name": " NOWAK ", "email": "a@x"})
create_contact(_store, {"first_name": "Jan", "last_name": "Kowalski", "email": "j@x"})
try:
    create_contact(_store, {"first_name": "X", "last_name": "Y", "email": "a@x"})
    assert False, "Duplikat email powinien rzucić ValueError"
except ValueError:
    pass

# READ
allc = list_contacts(_store)
assert len(allc) == 2 and {c["last_name"] for c in allc} == {"Nowak", "Kowalski"}

# FIND
res = find_by_lastname(_store, "ow")
assert [c["email"] for c in res] == ["a@x", "j@x"]

# UPDATE
c = update_contact(_store, "a@x", first_name=" Alicja ")
assert c["first_name"] == "Alicja" and _store["a@x"]["first_name"] == "Alicja"

# DELETE
assert delete_contact(_store, "j@x") is True
assert delete_contact(_store, "j@x") is False
assert set(_store) == {"a@x"}